In [14]:
# load all songs
import numpy as np
from tqdm import tqdm
import pandas as pd
name_csv = 'MassiveFemaledatabase.csv'
database = pd.read_csv(name_csv, error_bad_lines=False)


maxlen = 40
step = 1 # step size at every iteration
sentences = [] # list of sequences
next_chars = [] # list of next chracters that our model should predict
text = ""
for index, row in database['lyric'].iteritems():
    text = text + str(row).lower()
chars = sorted(list(set(text)))
#print('total chars:', len(chars))
# create a dictionary mapping chracter-to-index
char_indices = dict((c, i) for i, c in enumerate(chars))
# create a dictionary mapping index-to-chracter
indices_char = dict((i, c) for i, c in enumerate(chars))
# cut the text into sequences


for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
        
print ('nb sequences ', len(sentences))
for i, sentence in enumerate(tqdm(sentences)):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
        y[i, char_indices[next_chars[i]]] = 1


b'Skipping line 693: expected 4 fields, saw 5\nSkipping line 752: expected 4 fields, saw 5\nSkipping line 942: expected 4 fields, saw 5\nSkipping line 955: expected 4 fields, saw 5\n'
  1%|          | 11456/1703479 [00:00<00:30, 56353.01it/s]

nb sequences  1703479


100%|██████████| 1703479/1703479 [00:29<00:00, 57445.52it/s]


In [ ]:
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
from keras.models import Sequential
from keras.callbacks import LambdaCallback
from keras.layers import LSTM , Dense, Activation
from keras.optimizers import RMSprop
import keras
import numpy as np
import random
import sys
import io
import re
from keras.models import load_model


def search_last_model():
    max_epoch = 0
    for file in os.listdir("./"):
        if file.endswith(".h5"):
            epoch = int(re.findall('\d+', file )[0])
            if epoch > max_epoch :
                max_epoch = epoch
                max_epoch_file = file
        else :
            max_epoch_file = None
    if max_epoch_file != None:
        model = load_model(max_epoch_file)
        print ("Load Mode =", max_epoch_file)
    else :
        # create sequential network, because we are passing activations
        # down the network
        model = Sequential()
        optimizer = keras.optimizers.RMSprop(lr=0.01)
        # add LSTM layer
        model.add(LSTM(128, input_shape=(maxlen, len(chars))))
        # add Softmax layer to output one character
        model.add(Dense(len(chars)))
        model.add(Activation('softmax'))
        # compile the model and pick the loss and optimizer
        model.compile(loss='categorical_crossentropy', optimizer=optimizer )
        # train the model        
        
    return model , max_epoch
            
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


with open ('rpt.txt','w') as rpt:
    rpt.write('epoch,lyric\n')

def on_epoch_end(epoch, _ ):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(src) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = src[start_index: start_index + maxlen]
        print ("SRC =" , src , "\n")
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        for i in range(gen_length):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.
            #print (x_pred)
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char
            generated = generated + next_char
            sys.stdout.write(next_char)
            sys.stdout.flush()
        with open ('rpt.txt','a') as rpt:
            rpt.write("epoch = {}\n".format(epoch))
            rpt.write("diversity = {}\n".format(diversity))
            rpt.write("\n")
            for buf in generated:
                rpt.write(buf)
            rpt.write("\n")
            rpt.write("\n")
            rpt.write("\n")
        print()

src = 'i wake up in the dream'
gen_length = 400
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
checkpoint = keras.callbacks.ModelCheckpoint('model{epoch:08d}.h5', period=1, verbose=1) 

model , last_epoch = search_last_model()
optimizer = keras.optimizers.RMSprop(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer )
model.fit(x, y, batch_size=128, initial_epoch=last_epoch , epochs=50,callbacks=[print_callback,checkpoint])


Epoch 1/50
 369280/1703479 [=====>........................] - ETA: 6:51 - loss: 2.2044

In [ ]:
from keras.models import load_model
def load_model_gen():
    start_index = random.randint(0, len(src) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = src[start_index: start_index + maxlen]
        print ("SRC =" , src , "\n")
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        for i in range(gen_length):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.
            #print (x_pred)
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char
            generated = generated + next_char
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()